In [1]:
import pandas as pd
import pandas.io.sql as sqlio
import psycopg2
import typing

from configparser import ConfigParser
from psycopg2 import sql
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
from psycopg2.extras import execute_values


from database_functions import get_config, cryptodb_connection

In [5]:
#GLOBALS
name_Database = "crypto"
sqlCreateDatabase = f"create database {name_Database};"


In [6]:
#check if connection works
with cryptodb_connection() as db:
        with db.conn.cursor() as curs:
            curs.execute('SELECT version()')
            db_version = curs.fetchone()
            print(db_version)

('PostgreSQL 15.0, compiled by Visual C++ build 1914, 64-bit',)


In [7]:
#create Database
config = get_config(filename="database.ini", section="crypto")
con = psycopg2.connect(**config)
con.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)

# Obtain a DB Cursor
cursor          = con.cursor()

# Create database
try:
    cursor.execute(sqlCreateDatabase)
except Exception as e:
    print(f"""
                Following exception occurred: 
                    {e}
        """)


                Following exception occurred: 
                    database "crypto" already exists

        


In [21]:
#Create TABLES
with cryptodb_connection() as db:
        with db.conn.cursor() as curs:
            
            #create crypto_timeseries table
            curs.execute(
                            """ 
                                CREATE TABLE IF NOT EXISTS crypto_timeseries
                                (   
                                    index               SERIAL NOT NULL,
                                    timestamp           TIMESTAMP NOT NULL,
                                    id                  VARCHAR NOT NULL,
                                    rank                INT NOT NUll,
                                    symbol              VARCHAR NOT NULL,
                                    name                VARCHAR NOT NULL,
                                    supply              FLOAT,
                                    maxsupply           FLOAT,
                                    marketcapusd        FLOAT,
                                    volumeusd24hr       FLOAT,
                                    priceusd            FLOAT,
                                    changepercent24hr   FLOAT,
                                    vwap24hr            FLOAT,
                                    explorer            VARCHAR
                                );	
                            """
                        )
            
            #fecth tables in database
            curs.execute("select relname from pg_class where relkind='r' and relname !~ '^(pg_|sql_)';")
            print(curs.fetchall())

[('crypto_timeseries',)]
